In [8]:
import boto3
import json
import base64

# Create an STS client in application account using sts vpc endpoint at us-east-1

sts_client = boto3.client(
    'sts',
    region_name='us-east-1',
    endpoint_url='https://sts.us-east-1.amazonaws.com'
)

# Assume the role from ML account
assumed_role_object = sts_client.assume_role(
    RoleArn="arn:aws:iam::5931xxxxxxxx:role/account-a-account-b-assume-role",
    RoleSessionName="cross_account_lambda_invocation",
)


In [9]:
# Create a new session using the assumed role credentials
assumed_role_session = boto3.Session(
    aws_access_key_id=assumed_role_object['Credentials']['AccessKeyId'],
    aws_secret_access_key=assumed_role_object['Credentials']['SecretAccessKey'],
    aws_session_token=assumed_role_object['Credentials']['SessionToken']
)

# Create a Lambda client using the assumed role session and specify the desired region (us-east-1 in this example)
lambda_client = assumed_role_session.client('lambda', region_name='us-east-1')

In [10]:
def invoke_lambda_with_response_stream(function_name, payload, lambda_client):
    lambda_client = lambda_client

    response = lambda_client.invoke_with_response_stream(
        FunctionName=function_name,
        Payload=json.dumps(payload)
    )

    for event in response['EventStream']:
        if 'PayloadChunk' in event:
            chunk = event['PayloadChunk']['Payload'].decode('utf-8')
            print(chunk, end='', flush=True)
        elif 'InvokeComplete' in event:
            if 'ErrorCode' in event['InvokeComplete']:
                error_code = event['InvokeComplete']['ErrorCode']
                error_details = event['InvokeComplete']['ErrorDetails']
                print(f"\nError: {error_code} - {error_details}")
            else:
                print("\nInvocation completed successfully.")

In [11]:
# Example usage
function_name = 'lambdastreaming-NewStreamingFunction-dBu5NofsG2BE'

messages = [
  {"role": "user", "content": "What is AWS?"},
]

payload_body={
    "anthropic_version": "bedrock-2023-05-31",
    "messages": messages, 
    "max_tokens": 300, 
    "temperature": 0.1, 
    "top_p": 0.9}

payload = {
    'modelId': 'anthropic.claude-3-sonnet-20240229-v1:0',
    'body': payload_body
}



In [12]:
invoke_lambda_with_response_stream(function_name, payload, lambda_client)

AWS stands for Amazon Web Services. It is a comprehensive cloud computing platform provided by Amazon, offering a broad set of global cloud-based products and services. AWS allows individuals, companies, and governments to access a virtual cluster of computers, storage, databases, and a suite of application services over the internet.

Some key features and services offered by AWS include:

1. Elastic Compute Cloud (EC2): Provides scalable virtual servers in the cloud.
2. Simple Storage Service (S3): Offers scalable object storage for data backup, archiving, and applications.
3. Relational Database Service (RDS): Provides managed relational databases like MySQL, PostgreSQL, Oracle, and SQL Server.
4. Lambda: Allows running code without provisioning or managing servers (serverless computing).
5. Elastic Load Balancing: Automatically distributes incoming application traffic across multiple targets.
6. CloudFront: A content delivery network (CDN) service for low-latency distribution of da